# dAiv AI_Competition[2024]_Pro Baseline for PyTorch

## Import Libraries

In [1]:
from os import path, rename, mkdir, listdir

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, utils, models
from torchvision import transforms

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

datasets.utils.tqdm = tqdm
%matplotlib inline

### Check GPU Availability

In [2]:
!nvidia-smi

Sun Oct 27 14:59:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:04:00.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |    679MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   

In [3]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 7

device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
print("INFO: Using device -", device)

INFO: Using device - cuda


## Load DataSets

In [4]:
from typing import Callable, Optional
from sklearn.model_selection import train_test_split


class ImageDataset(datasets.ImageFolder):
    download_url = "https://daiv-cnu.duckdns.org/contest/ai_competition[2024]_pro/dataset/archive.zip"
    random_state = 20241028

    def __init__(
            self, root: str, force_download: bool = True,
            train: bool = False, valid: bool = False, split_ratio: float = 0.8,
            test: bool = False, unlabeled: bool = False,
            transform: Optional[Callable] = None, target_transform: Optional[Callable] = None
    ):
        self.download(root, force=force_download)  # Download Dataset from server

        if train or valid:  # Set-up directory
            root = path.join(root, "train")
        else:
            root = path.join(root, "test" if test else "unlabeled" if unlabeled else None)

        # Initialize ImageFolder
        super().__init__(root=root, transform=transform, target_transform=target_transform)

        if train or valid:  # Split Train and Validation Set
            seperated = train_test_split(
                self.samples, self.targets, test_size=1-split_ratio, stratify=self.targets, random_state=self.random_state
            )
            self.samples, self.targets = (seperated[0], seperated[2]) if train else (seperated[1], seperated[3])
            self.imgs = self.samples

    @classmethod
    def download(cls, root: str, force: bool = False):
        if force or not path.isfile(path.join(root, "archive.zip")):
            # Download and Extract Dataset
            datasets.utils.download_and_extract_archive(cls.download_url, download_root=root, extract_root=root, filename="archive.zip")
            
            # Arrange Dataset Directory
            for target_dir in [path.join(root, "test"), path.join(root, "unlabeled")]:
                for file in listdir(target_dir):
                    mkdir(path.join(target_dir, file.replace(".jpg", "")))
                    rename(path.join(target_dir, file), path.join(target_dir, file.replace(".jpg", ""), file))

            print("INFO: Dataset archive downloaded and extracted.")
        else:
            print("INFO: Dataset archive found in the root directory. Skipping download.")

### Dataset Initialization

In [5]:
# Image Resizing and Tensor Conversion
IMG_SIZE = (256, 256)
IMG_NORM = dict(  # ImageNet Normalization
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

resizer = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize Image
    transforms.ToTensor(),  # Convert Image to Tensor
    transforms.Normalize(**IMG_NORM)  # Normalization
])

In [6]:
DATA_ROOT = path.join(".", "data")

train_dataset = ImageDataset(root=DATA_ROOT, force_download=False, train=True, transform=resizer)
valid_dataset = ImageDataset(root=DATA_ROOT, force_download=False, valid=True, transform=resizer)

test_dataset = ImageDataset(root=DATA_ROOT, force_download=False, test=True, transform=resizer)
unlabeled_dataset = ImageDataset(root=DATA_ROOT, force_download=False, unlabeled=True, transform=resizer)

print(f"INFO: Dataset loaded successfully. Number of samples - Train({len(train_dataset)}), Valid({len(valid_dataset)}), Test({len(test_dataset)}), Unlabeled({len(unlabeled_dataset)})")

INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset loaded successfully. Number of samples - Train(7478), Valid(1870), Test(1110), Unlabeled(380)


## Data Augmentation if needed

In [7]:
ROTATE_ANGLE = 20
COLOR_TRANSFORM = 0.1

In [8]:
augmenter = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(ROTATE_ANGLE),
    transforms.ColorJitter(
        brightness=COLOR_TRANSFORM, contrast=COLOR_TRANSFORM,
        saturation=COLOR_TRANSFORM, hue=COLOR_TRANSFORM
    ),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    resizer
])

In [9]:
train_dataset = ImageDataset(root=DATA_ROOT, force_download=False, train=True, transform=augmenter)

print(f"INFO: Train dataset has been overridden with augmented state. Number of samples - Train({len(train_dataset)})")

INFO: Dataset archive found in the root directory. Skipping download.
INFO: Train dataset has been overridden with augmented state. Number of samples - Train(7478)


In [10]:
unlabeled_dataset = ImageDataset(root=DATA_ROOT, force_download=False, unlabeled=True, transform=augmenter)

print(f"INFO: Unlabeled dataset has been overridden with augmented state. Number of samples - Unlabeled({len(unlabeled_dataset)})")

INFO: Dataset archive found in the root directory. Skipping download.
INFO: Unlabeled dataset has been overridden with augmented state. Number of samples - Unlabeled(380)


## DataLoader

In [11]:
# Set Batch Size
BATCH_SIZE = 128

In [12]:
MULTI_PROCESSING = True  # Set False if DataLoader is causing issues

from platform import system
if MULTI_PROCESSING and system() != "Windows":  # Multiprocess data loading is not supported on Windows
    import multiprocessing
    cpu_cores = multiprocessing.cpu_count()
    print(f"INFO: Number of CPU cores - {cpu_cores}")
else:
    cpu_cores = 0
    print("INFO: Using DataLoader without multi-processing.")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=cpu_cores)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=cpu_cores)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=cpu_cores)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=cpu_cores)

INFO: Number of CPU cores - 48


In [13]:
# Image Visualizer
def imshow(image_list, mean=IMG_NORM['mean'], std=IMG_NORM['std']):
    np_image = np.array(image_list).transpose((1, 2, 0))
    de_norm_image = np_image * std + mean
    plt.figure(figsize=(10, 10))
    plt.imshow(de_norm_image)

In [14]:
# images, targets = next(iter(train_loader))
# grid_images = utils.make_grid(images, nrow=8, padding=10)
# imshow(grid_images)

In [15]:
# images, _ = next(iter(unlabeled_loader))
# grid_images = utils.make_grid(images, nrow=8, padding=10)
# imshow(grid_images)

## Define Model

In [16]:
class SecondMaxLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.clone()
        max_val, max_idx = torch.max(x, dim=1, keepdim=True)
        x.scatter_(1, max_idx, 1e-12)
        return x

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, d_k, d_v, h):
        super(MultiHeadAttention, self).__init__()
        self.h = h
        self.d_k = d_k
        self.d_v = d_v

        self.w_q = nn.Linear(d_model, h * d_k, bias=False)
        self.w_k = nn.Linear(d_model, h * d_k, bias=False)
        self.w_v = nn.Linear(d_model, h * d_v, bias=False)
        self.w_o = nn.Linear(h * d_v, d_model, bias=False)
        self.attention = ScaledDotProductAttention(d_k)

    def _split_into_heads(self, *xs):
        # x : [BATCH * SEQ_LEN * D_MODEL] -> [BATCH * H * SEQ_LEN * D]
        return [x.view(x.size(0), x.size(1), self.h, -1).transpose(1, 2) for x in xs]

    def forward(self, q, k, v, mask=None):
        # q, k, v : [BATCH * SEQ_LEN * D_MODEL]

        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)
        q, k, v = self._split_into_heads(q, k, v)  # -> q, k, v : [BATCH * H * SEQ_LEN * D]

        x = self.attention(q, k, v, mask)
        x = x.transpose(1, 2).reshape(x.size(0), x.size(2), -1)  # -> x : [BATCH * SEQ_LEN * D_MODEL]
        x = self.w_o(x)
        return x

In [18]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = d_k ** -0.5

    def forward(self, q, k, v, mask):
        # q, k, v : [BATCH * H * SEQ_LEN * D_K(D_V)]

        x = torch.matmul(q, k.transpose(-2, -1))  # -> x : BATCH * H * SEQ_LEN * SEQ_LEN

        x = x if mask is None else x.masked_fill(mask, float('-inf'))
        x = torch.matmul(torch.softmax(self.scale * x, dim=-1), v)
        return x

In [19]:
from gradient_reversal import GradientReversal

class ImageClassifier(nn.Module):
    def __init__(self, input_channel: int, output_channel: int, adaptive_pool_size: int, img_size: int, num_classes: int):
        super().__init__()
        self.multiple_output = False

        # 특징 추출기 (ResNet 백본)
        self.resnet = models.resnet34(pretrained=True)  # 필요에 따라 resnet50으로 변경 가능
        self.fc_size = self.resnet.fc.in_features  # FC 레이어의 입력 피처 수 (백본에 따라 동적으로 결정)

        # 마지막 평균 풀링과 FC 레이어 제거
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])  # [BATCH, CHANNELS, H, W]

        self.d_model = self.fc_size  # ResNet-34의 경우 512, ResNet-50의 경우 2048

        # Multi-Head Attention 파라미터 설정
        self.d_k = self.d_v = 64
        self.h = self.d_model // self.d_k  # 헤드 수를 동적으로 설정

        # 헤드 수가 정수인지 확인
        assert self.d_model % self.h == 0, "d_model은 h로 나누어 떨어져야 합니다."

        # Multi-Head Attention 레이어 추가
        self.attention_layer = MultiHeadAttention(d_model=self.d_model, d_k=self.d_k, d_v=self.d_v, h=self.h)

        # 어댑티브 평균 풀링으로 고정된 크기의 출력 생성
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # 도메인 분류기 - 복잡한 도메인 분류를 위한 다층 퍼셉트론(MLP)
        self.domain_classifier = nn.Sequential(
            nn.Linear(self.d_model, adaptive_pool_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(adaptive_pool_size, adaptive_pool_size // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(adaptive_pool_size // 2, 1)
        )

        # 출력 레이어 - 최종 예측을 위한 메인 분류기
        self.classifier = nn.Linear(self.d_model, num_classes)

        # 멀티라벨 분류를 위한 보조 분류기 (필요할 경우 사용)
        self.secondary = SecondMaxLayer()

    def toggle_multilabel(self, multi_label: bool | None = None):
        """싱글 라벨과 멀티 라벨 분류 사이를 전환하는 함수."""
        if isinstance(multi_label, bool):
            self.multiple_output = multi_label
        else:
            self.multiple_output = not self.multiple_output

    def forward(self, x, lambda_grl=1.0):
        # ResNet을 통한 특징 추출
        feature_map = self.resnet(x)  # [BATCH, D_MODEL, H, W]

        batch_size, d_model, H, W = feature_map.size()
        seq_len = H * W

        # 피처 맵을 시퀀스로 변환
        feature_seq = feature_map.view(batch_size, d_model, seq_len).permute(0, 2, 1)  # [BATCH, SEQ_LEN, D_MODEL]

        # Multi-Head Attention 적용
        attended_features = self.attention_layer(feature_seq, feature_seq, feature_seq)  # [BATCH, SEQ_LEN, D_MODEL]

        # 시퀀스를 피처 맵으로 복원
        attended_feature_map = attended_features.permute(0, 2, 1).view(batch_size, d_model, H, W)

        # 어댑티브 평균 풀링 적용
        pooled_features = self.adaptive_pool(attended_feature_map)  # [BATCH, D_MODEL, 1, 1]

        # 차원 축소
        extracted = pooled_features.view(batch_size, -1)  # [BATCH, D_MODEL]

        # 메인 분류기의 출력
        out = self.classifier(extracted)

        # GRL 적용 후 도메인 분류기 통과
        grl_layer = GradientReversal(lambda_grl)
        reversed_features = grl_layer(extracted)
        domain = self.domain_classifier(extracted)

        # 멀티라벨 출력이 활성화된 경우
        if self.multiple_output:
            return domain, out, self.secondary(out)

        # 도메인과 메인 분류기 출력 반환
        return domain, out

In [20]:
CLASS_LABELS = len(train_dataset.classes)

MODEL_PARAMS = dict(
    input_channel=3, output_channel=64, adaptive_pool_size=512,
    img_size=IMG_SIZE[0], num_classes=CLASS_LABELS
)

In [21]:
# Initialize Model
model = ImageClassifier(**MODEL_PARAMS)
model_id = 'DomainClassifier_ResNet34_Attention'
model.to(device)

/shared_hdd/rnoro5122/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/shared_hdd/rnoro5122/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ImageClassifier(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

## Training Loop

In [22]:
from IPython.display import display
import ipywidgets as widgets

# Interactive Loss Plot Update
def create_plot():
    losses = []

    # Enable Interactive Mode
    plt.ion()

    # Loss Plot Setting
    fig, ax = plt.subplots(figsize=(6, 2))
    line, = ax.plot(losses)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss")
    ax.set_title("Cross Entropy Loss")

    # Display Plot
    plot = widgets.Output()
    display(plot)

    def update_plot(new_loss):
        losses.append(new_loss.item())
        line.set_ydata(losses)
        line.set_xdata(range(len(losses)))
        ax.relim()
        ax.autoscale_view()
        with plot:
            plot.clear_output(wait=True)
            display(fig)

    return update_plot

In [23]:
# 모델 저장 및 불러오는 함수 정의
def save_checkpoint(epoch, model, optimizer, loss, PATH):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, PATH)
    print(f" Model saved.")

def load_checkpoint(PATH, model, optimizer):
    if path.isfile(PATH):
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        loss = checkpoint['loss']
        print(f"체크포인트 '{path.basename(PATH)}'에서 모델 로드 완료 (시작 에포크: {start_epoch})")
        return start_epoch, loss
    else:
        print(f"체크포인트 '{path.basename(PATH)}'를 찾을 수 없습니다. 새로 훈련을 시작합니다.")
        return 0, None

In [32]:
# Set Epoch Count
num_epochs = 40

In [33]:
# 학습률 조정 및 옵티마이저 선택
LEARNING_RATE = 0.0005
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

In [34]:
train_length, valid_length = map(len, (train_loader, valid_loader))

import itertools
from os import makedirs

# 손실 함수 정의
# criterion = nn.MultiLabelSoftMarginLoss()
criterion = nn.CrossEntropyLoss()
domain_criterion = nn.BCEWithLogitsLoss()  # 도메인 분류 손실 함수
domain_weight = 0.4 # 도메인 loss 가중치

PATH = path.join('checkpoints', f"{model_id}_checkpoint.pt.tar") # 모델 체크포인트 저장 경로
# 해당 경로에 폴더가 없을 경우 폴더 생성
makedirs('checkpoints', exist_ok=True)
save_cycle = 5

# 체크포인트 로드
start_epoch, _ = load_checkpoint(PATH, model, optimizer)

epochs = tqdm(range(start_epoch, num_epochs), desc="Running Epochs")
with (tqdm(total=train_length, desc="Training") as train_progress,
      tqdm(total=valid_length, desc="Validation") as valid_progress):  # Set up Progress Bars
    update = create_plot()  # Create Loss Plot

    for epoch in epochs:
        train_progress.reset(total=train_length)
        valid_progress.reset(total=valid_length)

        # Training
        model.train()
        model.toggle_multilabel(False)

        # 무한 반복 이터레이터 생성
        unlabeled_iter = iter(itertools.cycle(unlabeled_loader))

        for i, (inputs_source, targets_source) in enumerate(train_loader):
            optimizer.zero_grad()

            # 소스 도메인 데이터 설정
            inputs_source = inputs_source.to(device, non_blocking=True)
            targets_source = targets_source.to(device, non_blocking=True)
            domain_label_source = torch.zeros(inputs_source.size(0), 1, device=device)  # 소스 도메인 레이블: 0

            # 타겟 도메인 데이터 가져오기
            inputs_target, _ = next(unlabeled_iter)
            inputs_target = inputs_target.to(device, non_blocking=True)
            domain_label_target = torch.ones(inputs_target.size(0), 1, device=device)  # 타겟 도메인 레이블: 1

            # lambda_grl 값 설정 (필요에 따라 조정하거나 스케줄링 가능)
            lambda_grl = 0.5

            # 모델 출력 계산
            # 소스 도메인 데이터에서 분류와 도메인 분류 출력
            domain_outputs_source, outputs_source = model(inputs_source, lambda_grl)
            # 타겟 도메인 데이터에서 도메인 분류 출력 (레이블 없음)
            domain_outputs_target, _ = model(inputs_target, lambda_grl)

            # 손실 함수 계산
            # 분류 손실 (소스 도메인 데이터에 대해서만)
            classification_loss = criterion(outputs_source, targets_source)

            # 도메인 분류 손실 (소스와 타겟 도메인 모두)
            domain_outputs = torch.cat([domain_outputs_source, domain_outputs_target], dim=0)
            domain_labels = torch.cat([domain_label_source, domain_label_target], dim=0)
            domain_classification_loss = domain_criterion(domain_outputs, domain_labels)

            # 총 손실 계산 (분류 손실 + 가중치를 적용한 도메인 분류 손실)
            loss = classification_loss + domain_weight * domain_classification_loss

            # 역전파 및 옵티마이저 스텝
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            # Progress Bar 업데이트 및 출력
            update(loss)
            train_progress.update(1)
            print(f"\rEpoch [{epoch+1:2}/{num_epochs}], Step [{i+1:2}/{train_length}], Loss: {loss.item():.6f}", end="")

        val_acc, val_loss, val_domain_acc, total_batches = 0, 0, 0, 0

        # Validation
        model.eval()
        model.toggle_multilabel(False)
        with torch.no_grad():
            for inputs, targets in valid_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                domain_outputs, outputs = model(inputs)

                # 분류 손실 계산
                val_loss += criterion(outputs, targets).item() / valid_length
                val_acc += (torch.max(outputs, 1)[1] == targets.data).sum().item() / len(valid_dataset)

                # 도메인 분류기 정확도 계산
                domain_preds = (domain_outputs >= 0.5).float()  # 0.5 미만일 때 소스 도메인(0)으로 예측
                domain_labels = torch.zeros(inputs.size(0), 1).to(device)  # Validation 데이터는 소스 도메인으로 가정
                val_domain_acc += (domain_preds == domain_labels).sum().item()  # 정확도 누적
                total_batches += inputs.size(0)  # 배치 수 누적

                valid_progress.update(1)

        # 최종 도메인 분류 정확도 계산
        val_domain_acc /= total_batches  # 배치 수로 나눠 평균 계산

        print(f"\rEpoch [{epoch+1:2}/{num_epochs}], Step [{train_length}/{train_length}], Loss: {loss.item():.6f}, Valid Acc: {val_acc:.6%}, Valid Loss: {val_loss:.6f}, Domain Acc: {val_domain_acc:.6%}", end="\n" if (epoch+1) % save_cycle == 0 or (epoch+1) == num_epochs else "")

        # save_cycle마다 모델 저장
        if (epoch + 1) % save_cycle == 0:
            save_checkpoint(epoch, model, optimizer, loss.item(), PATH)

/tmp/ipykernel_41096/3120847572.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


체크포인트 'DomainClassifier_ResNet34_Attention_checkpoint.pt.tar'에서 모델 로드 완료 (시작 에포크: 20)


Running Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validation:   0%|          | 0/15 [00:00<?, ?it/s]

Output()

Epoch [22/40], Step [22/59], Loss: 0.216517, Valid Acc: 84.438503%, Valid Loss: 0.625685, Domain Acc: 87.967914%

KeyboardInterrupt: 

In [27]:
if not path.isdir(path.join(".", "models")):
    mkdir(path.join(".", "models"))

# Model Save
save_path = path.join(".", "models", f"{model_id}.pt")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Model saved to ./models/DomainClassifier_ResNet34_Attention.pt


# Model Evaluation

In [28]:
# Load Model

model = ImageClassifier(**MODEL_PARAMS)
model.load_state_dict(torch.load(path.join(".", "models", f"{model_id}.pt")))
model.to(device)

/tmp/ipykernel_41096/1017062602.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path.join(".", "models", f"{model_id}.pt")))


ImageClassifier(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [29]:
import torch.nn.functional as F

results = dict(id=[], label1=[], label2=[])
test_length = len(test_dataset)
output_threshold = 0.3 # outputs2에 대한 임계값 설정
domain_threshold = 0.5

model.eval()
model.toggle_multilabel(True)
with torch.no_grad():
    total_single = 0
    total_multi = 0
    for inputs, ids in tqdm(test_loader):
        inputs = inputs.to(device)
        domain, outputs1, outputs2 = model(inputs)
        domain = torch.sigmoid(domain)

        # outputs2에 소프트맥스 적용
        outputs2_prob = F.softmax(outputs2, dim=1)  # 각 클래스에 대한 확률 계산

        print(domain)

        # label1 처리
        preds1 = []
        preds2 = []
        for i in range(domain.size(0)):
            if domain[i].item() <= domain_threshold:
                # 소스 도메인: 단일 레이블 (-1)
                preds1.append(-1)
                # outputs1에 기반하여 preds2 설정
                preds2.append(torch.max(outputs1[i], 0)[1].item())
                total_single += 1
            else:
                # 타겟 도메인: 멀티레이블 추론 수행
                max_val2, max_idx2 = torch.max(outputs2_prob[i], 0)  # 확률 값으로 비교

                if max_val2.item() >= output_threshold:
                    # outputs2가 임계값 이상일 때, 멀티레이블 추론
                    preds1.append(torch.max(outputs1[i], 0)[1].item())
                    preds2.append(max_idx2.item())
                    total_multi += 1
                else:
                    # outputs2가 임계값 이하일 때, 단일 레이블로 간주
                    preds1.append(-1)  # 단일 레이블 (-1)
                    preds2.append(torch.max(outputs1[i], 0)[1].item())
                    total_single += 1

        results['id'] += [test_dataset.classes[i] for i in ids]
        results['label1'] += preds1
        results['label2'] += preds2

    print(f"single label: {total_single}, multi label: {total_multi}")

  0%|          | 0/9 [00:00<?, ?it/s]

tensor([[9.6070e-01],
        [9.9750e-01],
        [5.3024e-03],
        [4.1568e-01],
        [1.7959e-02],
        [9.9222e-01],
        [9.9999e-01],
        [5.3684e-01],
        [2.7298e-01],
        [1.5319e-01],
        [9.9797e-01],
        [5.0119e-04],
        [9.4049e-03],
        [9.9975e-01],
        [3.1499e-03],
        [9.9957e-01],
        [2.1880e-04],
        [9.9984e-01],
        [1.2301e-04],
        [9.8573e-01],
        [2.5638e-05],
        [4.7385e-01],
        [9.9963e-01],
        [7.7852e-01],
        [3.7425e-02],
        [7.2238e-01],
        [4.6276e-04],
        [8.9778e-01],
        [5.9550e-01],
        [4.1353e-03],
        [4.9730e-02],
        [9.6081e-01],
        [9.9972e-01],
        [9.9995e-01],
        [9.3049e-01],
        [1.2588e-01],
        [9.6231e-01],
        [9.9174e-01],
        [9.4200e-01],
        [1.1464e-01],
        [3.8519e-03],
        [7.7869e-01],
        [7.3313e-01],
        [6.6303e-02],
        [9.9326e-01],
        [1

In [30]:
# Re-arrange Results
for i, labels in enumerate(zip(results['label1'], results['label2'])):
    results['label1'][i], results['label2'][i] = min(labels), max(labels)
    # results['label1'][i], results['label2'][i] = -1, results['label1'][i]  # 멀티 라벨 분류 안하고 그냥 '-1, 라벨'로 처리

results_df = pd.DataFrame(results)
results_df

,id,label1,label2
0,TEST_00000,-1,58
1,TEST_00001,83,116
2,TEST_00002,-1,92
3,TEST_00003,-1,16
4,TEST_00004,-1,81
...,...,...,...
1105,TEST_01105,-1,2
1106,TEST_01106,-1,67
1107,TEST_01107,24,89
1108,TEST_01108,12,29


In [31]:
# Save Results
submission_dir = "submissions"
if not path.isdir(submission_dir):
    mkdir(submission_dir)

submit_file_path = path.join(submission_dir, f"{model_id}.csv")
results_df.to_csv(submit_file_path, index=False)
print("File saved to", submit_file_path)

File saved to submissions/DomainClassifier_ResNet34_Attention.csv
